In [ ]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd()))

from firewall_api import Firewall, LIKE, EQ, NOT
import json
import ipaddress
import csv

# Firewall Credentials# JSON File
#
#  {
#      "firewall_ip": "<FIREWALL_IP_ADDRESS>",
#      "username": "<USER_NAME>",
#      "port" : "<FIREWALL_PORT>"
#      "password": "<PASSWORD",
#
#  }
#
firewall_info = json.load(open("../Credentials/firewall_access.json"))
username = firewall_info["username"]
password = firewall_info["password"]
firewall_ip = firewall_info["firewall_ip"]
port = firewall_info["port"]

certificate_verify = False

firewall = Firewall(username=username, password=password, hostname="172.16.17.30", port=4444, certificate_verify=False)

print("Connection Test :: ", firewall.read("Login"))

In [ ]:
firewall = Firewall(
    username=username,
    password=password,
    hostname=firewall_ip,  # Use the variable instead of hardcoded IP
    # port=port,  # Convert port to integer
    certificate_verify=False,
    # timeout='30'  # Explicitly set timeout as integer
)
entity_name = "RW"
entity_type = "IPHost"

print("\n\n" + entity_type)

for index, item in enumerate(firewall.read(entity_type, entity_name, LIKE)["data"], start=1):
    print(f"{index:03} : {item}")

In [ ]:
firewall = Firewall(username, password, firewall_ip, port, certificate_verify)

with open("objects.csv", "r", encoding="UTF8", newline="") as csv_file:
    list_of_objects = list(csv.DictReader(csv_file))

In [ ]:
list_of_objects

In [ ]:
preffix = "MSS_"
entity_type = "IPHostGroup"
ip_host_group_name = preffix + "IPHost_Group"
entity_data = {
    "Name": ip_host_group_name,
    "IPFamily": "IPv4",
    "Description": "Creted by Integrity360 Sopohs Firewall API",
}
print(entity_data)
print("CREATE :: ", firewall.create(entity_type, entity_data), "\n")

entity_type = "FQDNHostGroup"
fqdn_group_name = preffix + "FQDN_Group"
entity_data = {
    "Name": fqdn_group_name,
    "IPFamily": "IPv4",
    "Description": "Creted by Integrity360 Sopohs Firewall API",
}
print(entity_data)
print("CREATE :: ", firewall.create(entity_type, entity_data), "\n")

In [ ]:
for item in list_of_objects:
    entity_data = {}

    if item["type"] == "ipv4":
        entity_data["IPFamily"] = "IPv4"  # IPv4 is Default settings
        entity_type = "IPHost"

        if "-" in item["data"]:
            entity_data = {
                "Name": preffix + "IPR_" + item["data"],
                "HostType": "IPRange",
                "StartIPAddress": item["data"].split("-")[0],
                "EndIPAddress": item["data"].split("-")[1],
            }
        else:
            network = ipaddress.IPv4Network(item["data"])

            if str(network.netmask) == "255.255.255.255":
                entity_data = {
                    "Name": preffix + "IPH_" + item["data"],
                    "HostType": "IP",
                    "IPAddress": str(network.network_address),
                }
            else:
                entity_data = {
                    "Name": preffix + "IPNW_" + item["data"],
                    "HostType": "Network",
                    "IPAddress": str(network.network_address),
                    "Subnet": str(network.netmask),
                }

        entity_data["HostGroupList"] = {"HostGroup": ip_host_group_name}

    elif item["type"] == "fqdn":
        entity_type = "FQDNHost"

        entity_data = {
            "Name": preffix + "fqdn_" + item["data"],
            "FQDN": item["data"],
            "FQDNHostGroupList": {"FQDNHostGroup": [fqdn_group_name]},
        }

    else:
        # Handle unsupported types
        print(f"Unsupported type: {item['type']} - Skipping item.")
        continue  # Skip to the next item in the list

    # Add common properties
    entity_data["Description"] = "Created by Ingertity360 Sophos Firewall API"

    # Debug output
    print(entity_data)

    # Perform the creation
    print("CREATE :: ", firewall.create(entity_type, entity_data))

In [ ]:
#   PRINT
firewall = Firewall(username, password, firewall_ip, port, certificate_verify)
entity_name = "MSS_"
entity_type = "IPHostGroup"
print("\n\n" + entity_type)
for item in enumerate(firewall.read(entity_type, entity_name, LIKE)["data"]):
    print(item)
entity_type = "IPHost"
print("\n\n" + entity_type)
for item in enumerate(firewall.read(entity_type, entity_name, LIKE)["data"]):
    print(item)
entity_name = "MSS_"
entity_type = "FQDNHostGroup"
print("\n\n" + entity_type)
for item in enumerate(firewall.read(entity_type, entity_name, LIKE)["data"]):
    print(item)
entity_type = "FQDNHost"
print("\n\n" + entity_type)
for item in enumerate(firewall.read(entity_type, entity_name, LIKE)["data"]):
    print(item)

In [ ]:
#
#   DELETE
# firewall = Firewall(username, password, firewall_ip, port, certificate_verify)
entity_name = "MSS_"
entity_type = "IPHostGroup"
print("DELETE :: ", firewall.delete(entity_type, entity_name, LIKE))
entity_type = "IPHost"
print("DELETE :: ", firewall.delete(entity_type, entity_name, LIKE))
entity_type = "FQDNHostGroup"
print("DELETE :: ", firewall.delete(entity_type, entity_name, LIKE))
entity_type = "FQDNHost"
print("DELETE :: ", firewall.delete(entity_type, entity_name, LIKE))